<center><img src="https://raw.githubusercontent.com/dimitreOliveira/MachineLearning/master/Kaggle/TensorFlow%202.0%20Question%20Answering/banner.png" width="1000"></center>

<br>
<center><h1>Using Tensorflow 2.0 with Bert on Natural Questions - (translated to TF2.0)</h1></center>
<br>

### This is a translated version of the baseline [script](https://www.kaggle.com/philculliton/using-tensorflow-2-0-w-bert-on-nq) from the Tensorflow team

#### I translated the script to the Tensorflow 2.0 version, this way we can take part in the TF2 prizes and may use the version to improve the work.

**A few notes:**
- If you want to keep using the flags you will have to use the "absl" lib (this is recommended by the TF team).
- Since we won't use it with the kernels, I removed most of the TPU related stuff to reduce complexity.
- I'm still translating the scripts.
- If you have experience with Tensorflow 2 or have any correction/improvement, please let me know.

In this notebook, we'll be using the Bert baseline for Tensorflow to create predictions for the Natural Questions test set. **Note that this uses a model that has already been pre-trained** - we're only doing inference here. A GPU is required, and this should take between 1-2 hours to run.

The original script can be found [here](https://github.com/google-research/language/blob/master/language/question_answering/bert_joint/run_nq.py).
The supporting modules were drawn from the [official Tensorflow model repository](https://github.com/tensorflow/models/tree/master/official). The bert-joint-baseline data is described [here](https://github.com/google-research/language/tree/master/language/question_answering/bert_joint).

**Note:** This baseline uses code that was migrated from TF1.x. Be aware that it contains use of tf.compat.v1, which is not permitted to be eligible for [TF2.0 prizes in this competition](https://www.kaggle.com/c/tensorflow2-question-answering/overview/prizes). It is intended to be used as a starting point, but we're excited to see how much better you can do using TF2.0!

In [1]:
import sys
sys.path.append("../bertjointbaseline")

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import numpy as np
import pandas as pd
import tensorflow as tf
# import tf2_0_baseline_w_bert as tf2baseline # old script
import tf2_0_baseline_w_bert_translated_to_tf2_0 as tf2baseline # my script
import bert_modeling as modeling
import bert_optimization as optimization
import bert_tokenization as tokenization
import json
import absl
import sys
# from tf2_0_baseline_w_bert import *

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# In this case, we've got some extra BERT model files under `/kaggle/input/bertjointbaseline`

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/bertjointbaseline.zip
../input/tensorflow2-question-answering/simplified-nq-test.jsonl
../input/tensorflow2-question-answering/sample_submission.csv
../input/tensorflow2-question-answering/simplified-nq-train.jsonl
../input/tensorflow2-question-answering/.ipynb_checkpoints/sample_submission-checkpoint.csv
../input/bertjointbaseline/bert_joint.ckpt.index
../input/bertjointbaseline/bert_joint.ckpt.data-00000-of-00001
../input/bertjointbaseline/bert_config.json
../input/bertjointbaseline/vocab-nq.txt
../input/bertjointbaseline/nq-train.tfrecords-00000-of-00001


#### Tensorflow flags are variables that can be passed around within the TF system. Every flag below has some context provided regarding what the flag is and how it's used.

#### Most of these can be changed as desired, with the exception of the Special Flags at the bottom, which _must_ stay as-is to work with the Kaggle back end.

In [2]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(absl.flags.FLAGS)

flags = absl.flags

flags.DEFINE_string(
    "bert_config_file", "../input/bertjointbaseline/bert_config.json",
    "The config json file corresponding to the pre-trained BERT model. "
    "This specifies the model architecture.")

flags.DEFINE_string("vocab_file", "../input/bertjointbaseline/vocab-nq.txt",
                    "The vocabulary file that the BERT model was trained on.")

flags.DEFINE_string(
    "output_dir", "../output/outdir",
    "The output directory where the model checkpoints will be written.")

flags.DEFINE_string("train_precomputed_file", None,
                    "Precomputed tf records for training.")

flags.DEFINE_integer("train_num_precomputed", None,
                     "Number of precomputed tf records for training.")

flags.DEFINE_string(
    "output_prediction_file", "../output/predictions.json",
    "Where to print predictions in NQ prediction format, to be passed to"
    "natural_questions.nq_eval.")

flags.DEFINE_string(
    "init_checkpoint", "../input/bertjointbaseline/bert_joint.ckpt",
    "Initial checkpoint (usually from a pre-trained BERT model).")

flags.DEFINE_bool(
    "do_lower_case", True,
    "Whether to lower case the input text. Should be True for uncased "
    "models and False for cased models.")

flags.DEFINE_integer(
    "max_seq_length", 384,
    "The maximum total input sequence length after WordPiece tokenization. "
    "Sequences longer than this will be truncated, and sequences shorter "
    "than this will be padded.")

flags.DEFINE_integer(
    "doc_stride", 128,
    "When splitting up a long document into chunks, how much stride to "
    "take between chunks.")

flags.DEFINE_integer(
    "max_query_length", 64,
    "The maximum number of tokens for the question. Questions longer than "
    "this will be truncated to this length.")

flags.DEFINE_bool("do_train", False, "Whether to run training.")

flags.DEFINE_bool("do_predict", True, "Whether to run eval on the dev set.")

flags.DEFINE_integer("train_batch_size", 32, "Total batch size for training.")

flags.DEFINE_integer("predict_batch_size", 8,
                     "Total batch size for predictions.")

flags.DEFINE_float("learning_rate", 5e-5, "The initial learning rate for Adam.")

flags.DEFINE_float("num_train_epochs", 3.0,
                   "Total number of training epochs to perform.")

flags.DEFINE_float(
    "warmup_proportion", 0.1,
    "Proportion of training to perform linear learning rate warmup for. "
    "E.g., 0.1 = 10% of training.")

flags.DEFINE_integer("save_checkpoints_steps", 1000,
                     "How often to save the model checkpoint.")

flags.DEFINE_integer("iterations_per_loop", 1000,
                     "How many steps to make in each estimator call.")

flags.DEFINE_integer(
    "n_best_size", 20,
    "The total number of n-best predictions to generate in the "
    "nbest_predictions.json output file.")

flags.DEFINE_integer(
    "verbosity", 1, "How verbose our error messages should be")

flags.DEFINE_integer(
    "max_answer_length", 30,
    "The maximum length of an answer that can be generated. This is needed "
    "because the start and end predictions are not conditioned on one another.")

flags.DEFINE_float(
    "include_unknowns", -1.0,
    "If positive, probability of including answers of type `UNKNOWN`.")

flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")
flags.DEFINE_bool("use_one_hot_embeddings", False, "Whether to use use_one_hot_embeddings")

absl.flags.DEFINE_string(
    "gcp_project", None,
    "[Optional] Project name for the Cloud TPU-enabled project. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

flags.DEFINE_bool(
    "verbose_logging", False,
    "If true, all of the warnings related to data processing will be printed. "
    "A number of warnings are expected for a normal NQ evaluation.")

flags.DEFINE_boolean(
    "skip_nested_contexts", True,
    "Completely ignore context that are not top level nodes in the page.")

flags.DEFINE_integer("task_id", 0,
                     "Train and dev shard to read from and write to.")

flags.DEFINE_integer("max_contexts", 48,
                     "Maximum number of contexts to output for an example.")

flags.DEFINE_integer(
    "max_position", 50,
    "Maximum context position for which to generate special tokens.")


## Special flags - do not change

flags.DEFINE_string(
    "predict_file", "../input/tensorflow2-question-answering/simplified-nq-test.jsonl",
    "NQ json for predictions. E.g., dev-v1.1.jsonl.gz or test-v1.1.jsonl.gz")
flags.DEFINE_boolean("logtostderr", True, "Logs to stderr")
flags.DEFINE_boolean("undefok", True, "it's okay to be undefined")
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string('HistoryManager.hist_file', '', 'kernel')

FLAGS = flags.FLAGS
FLAGS(sys.argv) # Parse the flags

['/users/liukanglong/.conda/envs/QA/lib/python3.7/site-packages/ipykernel_launcher.py']

#### Here, we:
1. Set up Bert
2. Read in the test set
3. Run it past the pre-built Bert model to create embeddings
4. Use those embeddings to make predictions
5. Write those predictions to `predictions.json`

Feel free to change the code below. Code for the `tf2baseline.*` functions is included in the `tf2_0_baseline_w_bert` utility script, and can be customized, whether by forking the utility script and updating it, or by creating your own non-`tf2baseline` versions in this kernel.

Note: the `tf2_0_baseline_w_bert` utility script contains code for training your own embeddings. Here that code is removed.

In [8]:
bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)

tf2baseline.validate_flags_or_throw(bert_config)
tf.io.gfile.makedirs(FLAGS.output_dir)

tokenizer = tokenization.FullTokenizer(
    vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

run_config = tf.estimator.RunConfig(
    model_dir=FLAGS.output_dir,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps)

num_train_steps = None
num_warmup_steps = None

model_fn = tf2baseline.model_fn_builder(
    bert_config=bert_config,
    init_checkpoint=FLAGS.init_checkpoint,
    learning_rate=FLAGS.learning_rate,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=FLAGS.use_tpu,
    use_one_hot_embeddings=FLAGS.use_one_hot_embeddings)

estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    config=run_config,
    params={'batch_size': FLAGS.train_batch_size})

if FLAGS.do_predict:
    if not FLAGS.output_prediction_file:
        raise ValueError(
            "--output_prediction_file must be defined in predict mode.")

    eval_examples = tf2baseline.read_nq_examples(
        input_file=FLAGS.predict_file, is_training=False)

    print("FLAGS.predict_file", FLAGS.predict_file)

    eval_writer = tf2baseline.FeatureWriter(
        filename=os.path.join(FLAGS.output_dir, "eval.tf_record"),
        is_training=False)
    eval_features = []


    def append_feature(feature):
        eval_features.append(feature)
        eval_writer.process_feature(feature)


    num_spans_to_ids = tf2baseline.convert_examples_to_features(
        examples=eval_examples,
        tokenizer=tokenizer,
        is_training=False,
        output_fn=append_feature)
    eval_writer.close()
    eval_filename = eval_writer.filename

    print("***** Running predictions *****")
    print(f"  Num orig examples = %d" % len(eval_examples))
    print(f"  Num split examples = %d" % len(eval_features))
    print(f"  Batch size = %d" % FLAGS.predict_batch_size)
    for spans, ids in num_spans_to_ids.items():
        print(f"  Num split into %d = %d" % (spans, len(ids)))

    predict_input_fn = tf2baseline.input_fn_builder(
        input_file=eval_filename,
        seq_length=FLAGS.max_seq_length,
        is_training=False,
        drop_remainder=False)

    print(eval_filename)

    # If running eval on the TPU, you will need to specify the number of steps.
    all_results = []

    for result in estimator.predict(
            predict_input_fn, yield_single_examples=True):
        if len(all_results) % 1000 == 0:
            print("Processing example: %d" % (len(all_results)))

        unique_id = int(result["unique_ids"])
        start_logits = [float(x) for x in result["start_logits"].flat]
        end_logits = [float(x) for x in result["end_logits"].flat]
        answer_type_logits = [float(x) for x in result["answer_type_logits"].flat]

        all_results.append(
            tf2baseline.RawResult(
                unique_id=unique_id,
                start_logits=start_logits,
                end_logits=end_logits,
                answer_type_logits=answer_type_logits))

    print("Going to candidates file")

    candidates_dict = tf2baseline.read_candidates(FLAGS.predict_file)

    print("setting up eval features")

    raw_dataset = tf.data.TFRecordDataset(eval_filename)
    eval_features = []
    for raw_record in raw_dataset:
        eval_features.append(tf.train.Example.FromString(raw_record.numpy()))

    print("compute_pred_dict")

    nq_pred_dict = tf2baseline.compute_pred_dict(candidates_dict, eval_features,
                                                 [r._asdict() for r in all_results])
    predictions_json = {"predictions": list(nq_pred_dict.values())}

    print("writing json")

    with tf.io.gfile.GFile(FLAGS.output_prediction_file, "w") as f:
        json.dump(predictions_json, f, indent=4)

INFO:tensorflow:Using config: {'_model_dir': 'outdir', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f98b604b310>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Reading: ../input/tensorflow2-question-answering/simplified-nq-test.jsonl
FLAGS.predict

INFO:tensorflow:  name = bert_model/encoder/layer_2/output/bias:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_2/output_layer_norm/gamma:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_2/output_layer_norm/beta:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_3/self_attention/query/kernel:0, shape = (1024, 1024)
INFO:tensorflow:  name = bert_model/encoder/layer_3/self_attention/query/bias:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_3/self_attention/key/kernel:0, shape = (1024, 1024)
INFO:tensorflow:  name = bert_model/encoder/layer_3/self_attention/key/bias:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_3/self_attention/value/kernel:0, shape = (1024, 1024)
INFO:tensorflow:  name = bert_model/encoder/layer_3/self_attention/value/bias:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_3/self_attention_output/kernel:0, shape = (1024, 1024)
INFO:tensorflow:  name =

INFO:tensorflow:  name = bert_model/encoder/layer_8/self_attention/key/kernel:0, shape = (1024, 1024)
INFO:tensorflow:  name = bert_model/encoder/layer_8/self_attention/key/bias:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_8/self_attention/value/kernel:0, shape = (1024, 1024)
INFO:tensorflow:  name = bert_model/encoder/layer_8/self_attention/value/bias:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_8/self_attention_output/kernel:0, shape = (1024, 1024)
INFO:tensorflow:  name = bert_model/encoder/layer_8/self_attention_output/bias:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_8/self_attention_layer_norm/gamma:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_8/self_attention_layer_norm/beta:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_8/intermediate/kernel:0, shape = (1024, 4096)
INFO:tensorflow:  name = bert_model/encoder/layer_8/intermediate/bias:0, shape = (4096,)
INFO:tens

INFO:tensorflow:  name = bert_model/encoder/layer_13/self_attention_output/kernel:0, shape = (1024, 1024)
INFO:tensorflow:  name = bert_model/encoder/layer_13/self_attention_output/bias:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_13/self_attention_layer_norm/gamma:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_13/self_attention_layer_norm/beta:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_13/intermediate/kernel:0, shape = (1024, 4096)
INFO:tensorflow:  name = bert_model/encoder/layer_13/intermediate/bias:0, shape = (4096,)
INFO:tensorflow:  name = bert_model/encoder/layer_13/output/kernel:0, shape = (4096, 1024)
INFO:tensorflow:  name = bert_model/encoder/layer_13/output/bias:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_13/output_layer_norm/gamma:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_13/output_layer_norm/beta:0, shape = (1024,)
INFO:tensorflow:  name = bert_model

INFO:tensorflow:  name = bert_model/encoder/layer_18/intermediate/kernel:0, shape = (1024, 4096)
INFO:tensorflow:  name = bert_model/encoder/layer_18/intermediate/bias:0, shape = (4096,)
INFO:tensorflow:  name = bert_model/encoder/layer_18/output/kernel:0, shape = (4096, 1024)
INFO:tensorflow:  name = bert_model/encoder/layer_18/output/bias:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_18/output_layer_norm/gamma:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_18/output_layer_norm/beta:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_19/self_attention/query/kernel:0, shape = (1024, 1024)
INFO:tensorflow:  name = bert_model/encoder/layer_19/self_attention/query/bias:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_19/self_attention/key/kernel:0, shape = (1024, 1024)
INFO:tensorflow:  name = bert_model/encoder/layer_19/self_attention/key/bias:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/l

INFO:tensorflow:  name = bert_model/encoder/layer_23/output_layer_norm/gamma:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/encoder/layer_23/output_layer_norm/beta:0, shape = (1024,)
INFO:tensorflow:  name = bert_model/pooler_transform/kernel:0, shape = (1024, 1024)
INFO:tensorflow:  name = bert_model/pooler_transform/bias:0, shape = (1024,)
INFO:tensorflow:  name = cls/nq/output_weights:0, shape = (2, 1024)
INFO:tensorflow:  name = cls/nq/output_bias:0, shape = (2,)
INFO:tensorflow:  name = answer_type_output_weights:0, shape = (5, 1024)
INFO:tensorflow:  name = answer_type_output_bias:0, shape = (5,)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Processing example: 0
Processing example: 1000
Processing example: 2000
Processing example: 3000
Processing example: 4000
Processing example: 5000
Processing example: 6000
Processing example: 7000
Processing example: 8000
P

NotFoundError: ../output/predictions.json; No such file or directory

#### Now, we turn `predictions.json` into a `submission.csv` file.

In [11]:
test_answers_df = pd.read_json("../output/predictions.json")

The Bert model produces a `confidence` score, which the Kaggle metric does not use. You, however, can use that score to determine which answers get submitted. See the limits commented out in `create_short_answer` and `create_long_answer` below for an example.

Values for `confidence` will range between `1.0` and `2.0`.

In [12]:
def create_short_answer(entry):
    # if entry["short_answers_score"] < 1.5:
    #     return ""
    
    answer = []    
    for short_answer in entry["short_answers"]:
        if short_answer["start_token"] > -1:
            answer.append(str(short_answer["start_token"]) + ":" + str(short_answer["end_token"]))
    if entry["yes_no_answer"] != "NONE":
        answer.append(entry["yes_no_answer"])
    return " ".join(answer)

def create_long_answer(entry):
   # if entry["long_answer_score"] < 1.5:
   # return ""

    answer = []
    if entry["long_answer"]["start_token"] > -1:
        answer.append(str(entry["long_answer"]["start_token"]) + ":" + str(entry["long_answer"]["end_token"]))
    return " ".join(answer)

In [13]:
test_answers_df["long_answer_score"] = test_answers_df["predictions"].apply(lambda q: q["long_answer_score"])
test_answers_df["short_answer_score"] = test_answers_df["predictions"].apply(lambda q: q["short_answers_score"])

In [14]:
test_answers_df["long_answer_score"].describe()

count    346.000000
mean       1.531710
std        0.172828
min        0.000000
25%        1.434122
50%        1.527620
75%        1.624831
max        2.159126
Name: long_answer_score, dtype: float64

An example of what each sample's answers look like in `prediction.json`:

In [15]:
test_answers_df.predictions.values[0]

{'example_id': '-1220107454853145579',
 'long_answer': {'start_token': 18,
  'end_token': 136,
  'start_byte': -1,
  'end_byte': -1},
 'long_answer_score': 1.4369705617427821,
 'short_answers': [{'start_token': 107,
   'end_token': 129,
   'start_byte': -1,
   'end_byte': -1}],
 'short_answers_score': 1.4369705617427821,
 'yes_no_answer': 'NONE'}

We re-format the JSON answers to match the requirements for submission.

In [16]:
test_answers_df["long_answer"] = test_answers_df["predictions"].apply(create_long_answer)
test_answers_df["short_answer"] = test_answers_df["predictions"].apply(create_short_answer)
test_answers_df["example_id"] = test_answers_df["predictions"].apply(lambda q: str(q["example_id"]))

long_answers = dict(zip(test_answers_df["example_id"], test_answers_df["long_answer"]))
short_answers = dict(zip(test_answers_df["example_id"], test_answers_df["short_answer"]))

Then we add them to our sample submission. Recall that each sample has both a `_long` and `_short` entry in the sample submission, one for each type of answer.

In [17]:
sample_submission = pd.read_csv("../input/tensorflow2-question-answering/sample_submission.csv")

long_prediction_strings = sample_submission[sample_submission["example_id"].str.contains("_long")].apply(lambda q: long_answers[q["example_id"].replace("_long", "")], axis=1)
short_prediction_strings = sample_submission[sample_submission["example_id"].str.contains("_short")].apply(lambda q: short_answers[q["example_id"].replace("_short", "")], axis=1)

sample_submission.loc[sample_submission["example_id"].str.contains("_long"), "PredictionString"] = long_prediction_strings
sample_submission.loc[sample_submission["example_id"].str.contains("_short"), "PredictionString"] = short_prediction_strings

And finally, we write out our submission!

In [18]:
sample_submission.to_csv("../output/submission.csv", index=False)
sample_submission.head()

,example_id,PredictionString
0,-1011141123527297803_long,277:339
1,-1011141123527297803_short,331:334
2,-1028916936938579349_long,42:321
3,-1028916936938579349_short,160:205
4,-1055197305756217938_long,16:221
